In [12]:
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaIoBaseDownload
import io, os, pickle

# If modifying these SCOPES, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def get_df_between_date(*, folder_id: str):
    """
    Get DataFrames from files within a specified date range.

    Args:
        folder_id (str): The ID of the Google Drive folder containing the data files.
        start_date (datetime | str): The start date of the range (inclusive).
                                    It can be either a datetime object or a string in the format '%Y%m%d'.
        end_date (datetime | str): The end date of the range (inclusive).
                                It can be either a datetime object or a string in the format '%Y%m%d'.

    Returns:
        list[pd.DataFrame]: A list of DataFrames read from files within the specified date range.
    """
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)

    results = service.files().list(
        q="'{}' in parents".format(folder_id), pageSize=10, fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])

    dfs = []
    if not items:
        print('No files found.')
    else:
        print('Files:')
        for item in items:
            print(u'{0} ({1})'.format(item['name'], item['id']))
            request = service.files().get_media(fileId=item['id'])
            fh = io.BytesIO()
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while done is False:
                status, done = downloader.next_chunk()
                print("Download %d%%." % int(status.progress() * 100))

            # The file has been downloaded into RAM, now read it into a DataFrame
            df = pd.read_csv(io.StringIO(fh.getvalue().decode()))
            dfs.append(df)

    return dfs

In [13]:
dfs = get_df_between_date(folder_id = "1JKRU7LguRL5gGcvNZ-fEOs3zpoH7IgrS")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1056585700984-1ubqptcqv3tepf8074p68h5h8viqi7li.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54660%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=1lgMrBWZAt2vkBfEl4vFXpwY8WY9Rj&access_type=offline
Files:
data.20201231_1200 (1kozeByivrc2faJBTE82in59ntm-8oEuc)
Download 100%.
data.20201230_1200 (1ZyGDWXVqSXfwo1DCheBlHKYiF219i9QY)
Download 100%.
data.20201229_1200 (1OpxGX66_MFWWvSLlreOrpvvN4eJg3ia-)
Download 100%.
data.20201228_1200 (1m12aXDnmXb67RvFJTGuTr6WRCe1iCs2h)
Download 100%.
data.20201224_1200 (1uhl6WSBY_TOF-_cER_5NGTCSDjEmLMQh)
Download 100%.
data.20201223_1200 (16yFeSvBHGlndMss4acSRCvRiBdQpdvkF)
Download 100%.
data.20201222_1200 (107PRglX4rYqVpV4wQPOLp3viQPKjAwfq)
Download 100%.
data.20201221_1200 (1mHIHT9UlKYUhWediMgsPkSjeo1pkzby-)
Download 100%.
data.20201218_1200 (1kcgM8Gr3pcRPdz0bs2snKZuz43Lw7WM6)
Download 100%

In [14]:
dfs[0]

,eqid,wt,today,tonight,tmwam,dn3sttmwmd,dn1,dn4x1,cftorrrelstd_open_0,liqlog_open_0,...,tr_md_1,tr_nt_0,tr_nt_1,tr_nt_12to16,tr_nt_17to21,tr_nt_1to3,tr_nt_4to6,tr_nt_7to11,tr_pm_1,tr_toxhr_0
0,2,0.000583,-0.005575,0.052731,-0.019832,0.002772,0.033508,-0.011956,-0.145521,-0.048219,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1,24,0.001439,-0.001190,-0.004553,-0.005368,-0.012527,-0.014156,-0.010522,-0.445552,1.600821,...,0.0,0.0,0.0,-0.000016,0.000000,0.000000,0.007493,0.015850,0.0,0.0
2,41,0.000072,-0.001752,0.018449,-0.004899,0.036978,0.020975,0.034163,-0.682080,-2.000973,...,0.0,0.0,0.0,0.003463,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,52,0.000097,-0.003835,0.001542,-0.012344,0.055697,-0.006965,0.041259,-0.712231,-1.719381,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,-0.003364,0.0,0.0
4,53,0.000394,-0.002562,0.002469,-0.005504,-0.027635,-0.017343,-0.014844,-0.047149,-0.414290,...,0.0,0.0,0.0,-0.005007,-0.005644,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3245,56398,0.000022,0.040447,0.002851,-0.040655,0.018863,0.006423,0.005516,-1.592990,-3.087824,...,0.0,0.0,0.0,-0.039029,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3246,56421,0.000056,-0.005768,-0.008373,0.025448,-0.005710,0.017248,0.001842,-1.605819,-2.231473,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.013959,0.0,0.0
3247,56423,0.000146,0.080217,-0.038563,0.024004,-0.145823,0.075937,-0.153258,-1.654203,-1.341729,...,0.0,0.0,0.0,-0.159353,0.000000,0.000000,0.000000,-0.182409,0.0,0.0
3248,56434,0.000096,0.057859,0.048236,0.046299,-0.094224,0.172437,-0.098412,-1.654203,-1.726393,...,0.0,0.0,0.0,0.000000,0.000000,0.019418,0.000000,0.013303,0.0,0.0
